In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# Training dataset 
file_path = 'Tr_DC_234_FAD.xlsx' 
data = pd.read_excel(file_path)

X = data[['Linear entropy', 'Tangle']].values
y = data['Label'].values

#Parameter grid for GridSearchCV
param_grid = {'C': np.arange(1,10,1)}

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5)
grid_search.fit(X, y)

# Get best C value from grid search
best_c = grid_search.best_params_['C']
print(f"The best C value is: {best_c}")

# Get the best estimator (model with the optimal C)
best_model = grid_search.best_estimator_

# Plotting decision boundary and data points
def plot_svm_boundary(X, y, model):
    plt.figure(figsize=(10, 8))

    #hyperplane
    w = model.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1)  
    yy = a * xx - (model.intercept_[0]) / w[1]

    
    xx_grid, yy_grid = np.meshgrid(np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 500),
                                   np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 500))
    Z = model.decision_function(np.c_[xx_grid.ravel(), yy_grid.ravel()])
    Z = Z.reshape(xx_grid.shape)

    
    plt.contourf(xx_grid, yy_grid, Z, levels=[Z.min(), 0, Z.max()], alpha=0.3, cmap=plt.cm.coolwarm)

    # Plot the AD data points and healthy data points
    plt.xlim(0.09, 0.4)
    plt.ylim(0.45, 0.88)
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='darkred', label='AD-Aβ (Train)', s=50, edgecolor='k')
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='darkblue', label='Healthy (Train)', s=50, edgecolor='k')

    # Plotting the hyperplane
    plt.plot(xx, yy, 'k-', linewidth=2) #, label='Decision Boundary'

    # Highlight test points and indicate correct or misclassified points
    y_pred = model.predict(X)
    correct = y_pred == y
    
    # Correctly classified points 
    plt.tick_params(labelsize=16)
    plt.scatter(X[correct, 0], X[correct, 1], facecolors='none', edgecolor='green', s=200, label='Correctly Classified')

    # Misclassified points 
    plt.scatter(X[~correct, 0], X[~correct, 1], marker='x', c='red', s=200, label='Misclassified')

    
    plt.xlabel('Linear Entropy', fontsize=18)
    plt.ylabel('Tangle', fontsize=18)
    plt.legend(fontsize="15")



plot_svm_boundary(X, y, best_model)

# Save plot
#fig = plt.gcf()
#fig.savefig('Train_SVM_Cortex1_FAD.svg')
#fig.savefig('Train_SVM_Cortex1_FAD.pdf')



